### 1. Update environment

In [1]:
!pip install --upgrade boto3 
!pip install --upgrade sagemaker 
!pip install nvidia-pyindex && pip install nvidia-cublas
!sudo yum update -y
!sudo yum install git-lfs git -y

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Loaded plugins: dkms-build-requires, extras_suggestions, langpacks, priorities,
              : update-motd, versionlock
amzn2-core                                               | 3.6 kB     00:00     
amzn2extra-docker                                        | 2.9 kB     00:00     
amzn2extra-kernel-5.10                                   | 3.0 kB     00:00     
amzn2extra-python3.8                                     | 2.9 kB     00:00     
centos-extras                                            | 2.9 kB     00:00     
copr:copr.fedorainfracloud.org:vbatts:shadow-utils-newxi | 3.3 kB     00:00     
https://download.docker.com/linux/centos/2/x86_64/stable/repodata/repomd.xml: [Errno 14] HTTPS Error 

In [2]:
import sagemaker
from sagemaker import image_uris
import boto3
import os
import time
import json

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:

if "cn-" in region:
    repository = "TheBloke/Llama-2-7b-chat-GGML" #todo:find one repo
    url_suffix = repository + ".git"
    repo = "https://www.wisemodel.cn/" + url_suffix
else:
    repository = "" #todo:find one repo
    # Select 7b or 13b model parameter
    repo = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf"
    # repo = "https://huggingface.co/TheBloke/Llama-2-7b-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_0.gguf"
model_id="llama-2-cpp"
s3_model_prefix =model_id
s3_code_prefix=model_id
bucket = "sagemaker-"+region+"-" + account_id  #specify your s3 dir to store model repo example-> s3://your-bucket-name/your-s3-dir
s3 = boto3.client('s3')
directory_name = model_id  #it's name of your folders
s3.put_object(Bucket=bucket, Key=(directory_name+'/'))



{'ResponseMetadata': {'RequestId': 'GWCAPDT60N447G85',
  'HostId': 'Nzy7sv4R4QDgK+TUSzzlrzBuOawprP956/jTFS1yXUh/CIR+veKCscM/v2o740yfvP2q3IXrACfcrnT2bEd0wA==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'Nzy7sv4R4QDgK+TUSzzlrzBuOawprP956/jTFS1yXUh/CIR+veKCscM/v2o740yfvP2q3IXrACfcrnT2bEd0wA==',
   'x-amz-request-id': 'GWCAPDT60N447G85',
   'date': 'Sun, 29 Oct 2023 09:22:56 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"d41d8cd98f00b204e9800998ecf8427e"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"',
 'ServerSideEncryption': 'AES256'}

In [4]:
!mkdir $model_id
s3_location = "s3://"+bucket + "/" + model_id + "/"
local_model_dir = "./" + model_id 

!echo $s3_location
!echo $repo
!echo $local_model_dir
!echo $model_id

s3://sagemaker-us-east-1-524648928792/llama-2-cpp/
https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf
./llama-2-cpp
llama-2-cpp


In [5]:
!wget $repo

--2023-10-29 09:22:56--  https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf
Resolving huggingface.co (huggingface.co)... 99.84.108.129, 99.84.108.70, 99.84.108.87, ...
Connecting to huggingface.co (huggingface.co)|99.84.108.129|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/8d/b1/8db1d1f73b4caa58e947ccbfe2fb27ac5e495c2ad8457ad299d15987aee3b520/eda2a15d532bea4ce6fc14d15c2b72638243396816252f73a94dceeb0429112f?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27llama-2-13b-chat.Q4_0.gguf%3B+filename%3D%22llama-2-13b-chat.Q4_0.gguf%22%3B&Expires=1698830576&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5ODgzMDU3Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy84ZC9iMS84ZGIxZDFmNzNiNGNhYTU4ZTk0N2NjYmZlMmZiMjdhYzVlNDk1YzJhZDg0NTdhZDI5OWQxNTk4N2FlZTNiNTIwL2VkYTJhMTVkNTMyYmVhNGNlNmZjMTRkMTVjMmI3MjYzODI0MzM5

In [6]:
!cd $local_model_dir && rm -rf ".ipynb_checkpoints"
!touch $local_model_dir/model
!tar czvf model.tar.gz $local_model_dir/*

./llama-2-cpp/model


In [7]:
s3_model = sess.upload_data("model.tar.gz", bucket, directory_name)
#!aws s3 sync $local_model_dir $s3_location

### 2. Make docker image
##### Notice: you can modify Dockfile to ADD different model weights.

In [8]:
!docker build -t llama2-cpp-python-sagemaker .

Sending build context to Docker daemon  7.366GB
Step 1/23 : ARG CUDA_IMAGE="12.1.1-devel-ubuntu22.04"
Step 2/23 : FROM nvidia/cuda:${CUDA_IMAGE}
 ---> 9df437e8fe97
Step 3/23 : ENV CUDA_DOCKER_ARCH=all
 ---> Using cache
 ---> e1d7f510939e
Step 4/23 : ENV LLAMA_CUBLAS=1
 ---> Using cache
 ---> 42bf1c7c7a1a
Step 5/23 : RUN apt update && apt install -y python3 python3-pip
 ---> Using cache
 ---> d74c987c8f2d
Step 6/23 : RUN python3 -m pip install --upgrade pip pytest cmake scikit-build setuptools fastapi uvicorn sse-starlette pydantic-settings
 ---> Using cache
 ---> 6187ef01693d
Step 7/23 : ARG DEBIAN_FRONTEND=noninteractive
 ---> Using cache
 ---> 835a0d11c871
Step 8/23 : RUN apt-get update     && apt-get  -y -f install build-essential libgeos-dev
 ---> Using cache
 ---> 871941465154
Step 9/23 : ARG FUNCTION_DIR="/opt/program"
 ---> Using cache
 ---> d379ac40a70b
Step 10/23 : WORKDIR ${FUNCTION_DIR}
 ---> Using cache
 ---> ff772b0def29
Step 11/23 : ADD requirements.txt ${FUNCTION_DIR}
 -

In [9]:
#AWS ECR Login,
!docker login -u AWS -p $(aws ecr get-login-password --region $region) https://{account_id}.dkr.ecr.{region}.amazonaws.com

#Create ECR Repo
!aws ecr create-repository --repository-name llama2-cpp-python-sagemaker --image-scanning-configuration scanOnPush=true --image-tag-mutability MUTABLE

#Tag Image
!docker tag llama2-cpp-python-sagemaker:latest {account_id}.dkr.ecr.{region}.amazonaws.com/llama2-cpp-python-sagemaker:latest

#Push Image to ECR
!docker push {account_id}.dkr.ecr.{region}.amazonaws.com/llama2-cpp-python-sagemaker:latest

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
{
    "repository": {
        "repositoryArn": "arn:aws:ecr:us-east-1:524648928792:repository/llama2-cpp-python-sagemaker",
        "registryId": "524648928792",
        "repositoryName": "llama2-cpp-python-sagemaker",
        "repositoryUri": "524648928792.dkr.ecr.us-east-1.amazonaws.com/llama2-cpp-python-sagemaker",
        "createdAt": 1698571902.0,
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": true
        },
        "encryptionConfiguration": {
            "encryptionType": "AES256"
        }
    }
}
The push refers to repository [524648928792.dkr.ecr.us-east-1.amazonaws.com/llama2-cpp-python-sagemaker]

5738d87

### 3. Deploy Sagemaker Endpoint(Execute this python code snippet in juypter notebook or ec2)


In [10]:
from sagemaker import get_execution_role
from sagemaker.model import Model
from sagemaker.predictor import Predictor
from sagemaker.session import Session
import boto3

#ECR URI
image_uri = account_id+'.dkr.ecr.'+region+'.amazonaws.com/llama2-cpp-python-sagemaker'
# This can be dummy model file
model_dir = s3_model
endpoint_name = 'pytorch-inference-llm-v1'
if "cn-" in region:
    instance_type='ml.g4dn.2xlarge'
else:
    instance_type='ml.g5.2xlarge'
# Create the SageMaker model instance
model = Model(
    image_uri=image_uri,
    role=sagemaker.get_execution_role(),
    model_data=model_dir
)


client = boto3.client('sagemaker')
try:
    response = client.delete_endpoint_config(EndpointConfigName=endpoint_name)
except:
    pass

model.deploy(
    instance_type=instance_type,
    initial_instance_count=1,
    endpoint_name = endpoint_name,
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
--------------!

#### Invoke Sagemaekr Endpoint


In [11]:
import boto3
import time

data = {"ask": "how to learn english?",
           "max_tokens" :"5000",
}
runtime_sagemaker_client = boto3.client(service_name="sagemaker-runtime")
#Please write your endpoint name which you want to use
endpoint_name = 'pytorch-inference-llm-v1'

body = json.dumps(data)

start = time.time()
response = runtime_sagemaker_client.invoke_endpoint(
    EndpointName = endpoint_name,
    ContentType  = "application/json",
    Body= body)

cost = time.time() - start     
result = response['Body'].read().decode('utf-8')

print('Response: ', result[12:-2])
print("\nCost Time:  %s seconds" % (cost))
print('Output Chars :', len(result[12:-3]))
print('Speed: {:.2f} Chars/s'.format(len(result)/float(cost)))

Response:  \n\nLearning English can be a challenging task, but with the right approach and resources, it can also be a rewarding experience. Here are some tips on how to learn English effectively:\n\n1. Set goals: Define your goals and what you want to achieve in learning English. This will help you stay motivated and focused.\n2. Immerse yourself in the language: Listen to English podcasts, watch English TV shows or movies, read English books or articles, and speak English with native speakers or language exchange partners.\n3. Practice consistently: Make learning English a regular part of your routine, whether it's daily or a few times a week. Consistency is key to improving your skills.\n4. Focus on grammar and vocabulary: Understanding the rules of English grammar and building your vocabulary are essential for effective communication.\n5. Use language learning apps: There are many apps available that can help you learn English, such as Duolingo, Babbel, and Rosetta Stone.\n6. Take 

#### 4.Clean Endpoint and config

In [12]:
# !aws sagemaker delete-endpoint --endpoint-name $endpoint_name
# !aws sagemaker delete-endpoint-config --endpoint-config-name $endpoint_name


#### Additional : Speed comparison with JumpStart endpoint with Llama2 7B Chat，for the other model deployed by JumpStart

In [13]:
# import boto3
# import time

# payload = {
#     "inputs": [[{"role": "user", "content": "how to learn english?"}]], 
#     "parameters": {"max_new_tokens": 1000, "top_p": 0.9, "temperature": 0.6}
# }
# endpoint_name = 'pytorch-inference-llm-v1'

# client = boto3.client("sagemaker-runtime")
# start = time.time()
# response = client.invoke_endpoint(
#     EndpointName=endpoint_name,
#     ContentType="application/json",
#     Body=json.dumps(payload),
#     CustomAttributes="accept_eula=true",
# )
# cost = time.time() - start     

# response = response["Body"].read().decode("utf8")
# print('Response: ', response[69:-11])
# print("Cost Time:  %s seconds" % (cost))
# print('Output Chars :', len(response[69:-11]))
# print('Speed: {:.2f} Chars/s'.format(len(response)/float(cost)))